In [1]:
import xcf
import numpy as np
import pandas as pd
import tensorflow as tf
import keras_nlp

2023-04-25 08:18:51.168801: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-25 08:18:51.847273: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [16]:
PRETRAINING_BATCH_SIZE = 32
AA_VOCAB = ['<PAD>','<START>','<END>','<MASK>','A','B','C','D','E','F','G','H','I','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']
MAX_SEQUENCE_LENGTH = 600 #Not counting <START> and <END> tokens
MASK_RATE = 0.25
PREDICTIONS_PER_SEQ = 32

In [13]:
data_prelim = xcf.load_data_from_db('data_pHpred_codetest', mode = 'clean')
data_prelim[['tokens', 'ids']] = data_prelim['sequence'].apply(xcf.tokenizers.tokenize_protein, args=(AA_VOCAB, MAX_SEQUENCE_LENGTH))
data_prelim['seq_as_words'] = data_prelim['sequence'].apply(lambda s: ' '.join([char for char in s]))
data_prelim.head()

,pH,sequence,len_seq,tokens,ids,seq_as_words
rcsb_id,,,,,,
4ZLN,7.5,DEYEIIETIGNGAYGVVSSARRRLTGQQVAIKKIPNAFDVVTNAKR...,341,"[<START>, D, E, Y, E, I, I, E, T, I, G, N, G, ...","[1, 7, 8, 27, 8, 12, 12, 8, 22, 12, 10, 16, 10...",D E Y E I I E T I G N G A Y G V V S S A R R R ...
4TWU,7.5,ANKTVVVTTILESPYVMMKKNHEMLEGNERYEGYCVDLAAEIAKHC...,262,"[<START>, A, N, K, T, V, V, V, T, T, I, L, E, ...","[1, 4, 16, 13, 22, 24, 24, 24, 22, 22, 12, 14,...",A N K T V V V T T I L E S P Y V M M K K N H E ...
4GRD,8.0,PAAAMFIVNTNVPRASVPDGFLSELTQQLAQATGKPPQYIAVHVVP...,117,"[<START>, P, A, A, A, M, F, I, V, N, T, N, V, ...","[1, 18, 4, 4, 4, 15, 9, 12, 24, 16, 22, 16, 24...",P A A A M F I V N T N V P R A S V P D G F L S ...
2ZRH,4.6,MPDIVNRKVEHVEIAAFENVDGLSSSTFLNDVILVHQGFPGISFSE...,368,"[<START>, M, P, D, I, V, N, R, K, V, E, H, V, ...","[1, 15, 18, 7, 12, 24, 16, 20, 13, 24, 8, 11, ...",M P D I V N R K V E H V E I A A F E N V D G L ...
2GAL,6.0,MNLKDKILGVAKELFIKNGYNATTTGEIVKLSESSKGNLYYHFKTK...,194,"[<START>, M, N, L, K, D, K, I, L, G, V, A, K, ...","[1, 15, 16, 14, 13, 7, 13, 12, 14, 10, 24, 4, ...",M N L K D K I L G V A K E L F I K N G Y N A T ...


In [18]:
ds_prelim = tf.data.Dataset.from_tensor_slices((np.array(data_prelim['seq_as_words'].values.tolist())))
ds_prelim_train, ds_prelim_val, ds_prelim_test = xcf.split_ds(ds_prelim)
ds_prelim_train = ds_prelim_train.batch(PRETRAINING_BATCH_SIZE)
ds_prelim_val = ds_prelim_val.batch(PRETRAINING_BATCH_SIZE)
ds_prelim_test = ds_prelim_test.batch(PRETRAINING_BATCH_SIZE)


In [21]:
tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary = AA_VOCAB,
    sequence_length = MAX_SEQUENCE_LENGTH,
    oov_token = 'X'
)
masker = keras_nlp.layers.MaskedLMMaskGenerator(
    vocabulary_size=tokenizer.vocabulary_size(),
    mask_selection_rate=MASK_RATE,
    mask_selection_length=PREDICTIONS_PER_SEQ,
    mask_token_id=tokenizer.token_to_id("<MASK>"),
)

def preprocess(ds_prelim_train):
    inputs = tokenizer(ds_prelim_train)
    outputs = masker(inputs)
    features = {
        "token_ids": outputs["token_ids"],
        "mask_positions": outputs["mask_positions"],
    }
    labels = outputs["mask_ids"]
    weights = outputs["mask_weights"]
    return features, labels, weights

ds_prelim_train = ds_prelim_train.map(preprocess)
ds_prelim_train.take(1).get_single_element()

2023-04-25 08:50:19.280076: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'masked_lm_mask_generator_1/RaggedGather/cond/zeros/Reshape/masked_lm_mask_generator_1/RaggedGather/strided_slice' with dtype int64
	 [[{{node masked_lm_mask_generator_1/RaggedGather/cond/zeros/Reshape/masked_lm_mask_generator_1/RaggedGather/strided_slice}}]]
2023-04-25 08:50:19.285253: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'masked_lm_mask_generator_1/RaggedGather/cond/cond/range/masked_lm_mask_generator_1/RaggedGather/sub_1' with dtype int64
	 [[{{node masked_lm_mask_generator_1/RaggedGather/cond/cond/range/masked_lm_mask_generator_1/R

({'token_ids': <tf.Tensor: shape=(32, 600), dtype=int32, numpy=
  array([[15, 20, 10, ...,  0,  0,  0],
         [21, 15, 24, ...,  0,  0,  0],
         [21,  7, 13, ...,  0,  0,  0],
         ...,
         [ 4,  4, 21, ...,  0,  0,  0],
         [15, 21, 14, ...,  0,  0,  0],
         [15, 16, 12, ...,  0,  0,  0]], dtype=int32)>,
  'mask_positions': <tf.Tensor: shape=(32, 32), dtype=int64, numpy=
  array([[ 15,  26,  27, ..., 294, 306, 315],
         [  4,   8,  11, ..., 161, 165, 166],
         [  5,   6,   9, ...,   0,   0,   0],
         ...,
         [ 13,  28,  75, ..., 432, 439, 448],
         [ 12,  16,  36, ..., 202, 205, 214],
         [ 35,  40,  46, ..., 155, 164, 167]])>},
 <tf.Tensor: shape=(32, 32), dtype=int32, numpy=
 array([[ 9, 12, 21, ..., 11, 14, 13],
        [21, 27, 14, ...,  7, 21, 21],
        [11, 14,  7, ...,  0,  0,  0],
        ...,
        [26, 12, 12, ..., 10, 26, 26],
        [19, 21, 12, ...,  4, 19, 25],
        [21,  4,  7, ..., 10, 14, 11]], dtype=i